In [1]:
import pandas as pd
import numpy as np 
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from IPython.display import Image 

In [2]:
playstoredata = pd.read_csv('googleplaystore.csv')
playstoredata.head()
print(playstoredata.dtypes)

App                object
Category           object
Rating            float64
Reviews            object
Size               object
Installs           object
Type               object
Price              object
Content Rating     object
Genres             object
Last Updated       object
Current Ver        object
Android Ver        object
dtype: object


In [3]:
print(playstoredata.isnull().sum())


App                  0
Category             0
Rating            1474
Reviews              0
Size                 0
Installs             0
Type                 1
Price                0
Content Rating       1
Genres               0
Last Updated         0
Current Ver          8
Android Ver          3
dtype: int64


In [4]:

playstoredata.dropna(axis =0,inplace = True)



In [5]:
def Install(installs):
    installs = installs.strip('+')
    installs = installs.replace(',','')
    return int(installs)


In [6]:
playstoredata['Installs'] = playstoredata['Installs'].apply(lambda ins:Install(ins))
playstoredata.dropna(axis =0,inplace = True)

In [7]:
def Size(size):
    if  'M' in size:
        size =size.replace('M','')
        size = float(size) * 1048576
        return(size)
    elif 'k' in size:
        size = size.replace('k','')
        size = float(size) * 1024
        return(size)
    else:
        return None



In [8]:
playstoredata['Size'] = playstoredata['Size'].apply(lambda siz: Size(siz))
playstoredata.dropna(axis =0,inplace = True)

In [9]:
category = pd.get_dummies(playstoredata['Category'],prefix='Ct')
Type = pd.get_dummies(playstoredata['Type'],prefix='ty')
content_rating = pd.get_dummies(playstoredata['Content Rating'],prefix='Cr')
Geners_rating = pd.get_dummies(playstoredata['Genres'],prefix = 'Grs')


In [10]:
playstoredata['Rating'] = playstoredata['Rating'].apply(lambda rat:round(rat))

In [11]:
print(playstoredata['Rating'].value_counts())

4    5399
5    1638
3     540
2     126
1      20
Name: Rating, dtype: int64


In [12]:
Target = playstoredata['Rating'].values
Reviews = playstoredata['Reviews']
Installs = playstoredata['Installs']

final_df = [category,Type,content_rating,Geners_rating,Reviews,Installs]
Concated_df = pd.concat(final_df,axis =1)


In [13]:
print(Concated_df.shape)


(7723, 155)


In [14]:
from sklearn.preprocessing import MinMaxScaler
MinMScaler = MinMaxScaler()
transoformed_df = MinMScaler.fit_transform(Concated_df)

In [15]:
CX_train,CX_test,Cy_train,Cy_test = train_test_split(transoformed_df,Target,random_state = 3,test_size = 0.2)

In [16]:

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import graphviz 
from sklearn import tree
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn import metrics

from sklearn.tree import export_graphviz
from sklearn.externals.six import StringIO  
import pydotplus


c:\users\user 38\appdata\local\programs\python\python37\lib\site-packages\sklearn\externals\six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)


In [22]:
import os
featurenames = Concated_df.columns
os.environ["PATH"] += os.pathsep + 'C:\\Program Files (x86)\\Graphviz2.38\\bin'

In [72]:
DecisionTree = DecisionTreeClassifier(criterion='entropy')
DecisionTree.fit(CX_train,Cy_train)
DecisionTree_Preds = DecisionTree.predict(CX_test)

In [25]:
accuracy_score(Cy_test,DecisionTree_Preds)

0.6614886731391586

In [58]:
dot_data = StringIO()

In [66]:
import graphviz 
from sklearn import tree
dotdata = tree.export_graphviz(DecisionTree, out_file=None,feature_names = Concated_df.columns,class_names=['1','2','3','4','5']) 
graph = graphviz.Source(dotdata) 
graph.render('Decision')

In [39]:
RadndomClassifier = RandomForestClassifier()
RadndomClassifier.fit(CX_train,Cy_train)
Random_preds = RadndomClassifier.predict(CX_test)
accuracy_score(Cy_test,Random_preds)


0.6912621359223301

In [40]:
RandomCls = RandomForestClassifier(n_estimators=1000,ccp_alpha=1)
RandomCls.fit(CX_train,Cy_train)
RandomCls_preds = RandomCls.predict(CX_test)
accuracy_score(Cy_test,RandomCls_preds)

0.7190938511326861

In [43]:

DTCost = DecisionTreeClassifier(max_depth=6 ,ccp_alpha =1,criterion='entropy')
DTCost.fit(CX_train,Cy_train)
DTcost_pred = DTCost.predict(CX_test)
accuracy_score(Cy_test,DTcost_pred)


0.7190938511326861

In [44]:
from sklearn.model_selection import KFold 
kfold = KFold(n_splits=5, random_state=2) 


for trainindex, testindex in kfold.split(transoformed_df):
    cv_CX_train, cv_CX_test = transoformed_df[trainindex], transoformed_df[testindex]
    cv_Cy_train, cv_Cy_test = Target[trainindex], Target[testindex]


c:\users\user 38\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


In [45]:
DClassifier = DecisionTreeClassifier(criterion='entropy')
DClassifier.fit(cv_CX_train,cv_Cy_train)
cv_Tree_preds = DClassifier.predict(cv_CX_test)
accuracy_score(cv_Cy_test,cv_Tree_preds)


0.6055699481865285

In [46]:
DCler = DecisionTreeClassifier(criterion='entropy',max_depth=5 ,ccp_alpha =1)
DCler.fit(cv_CX_train,cv_Cy_train)
DClr_preds = DCler.predict(cv_CX_test)
accuracy_score(cv_Cy_test,DClr_preds)


0.6813471502590673